In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import tqdm

In [2]:
hdf_file="v2_lagging_C28may_28may_2020_2021_ada.hdf"
df = pd.read_hdf(hdf_file)
df = df[:-180]
df

,trade_in_3h_usdtfutures_ADAUSDT_close_price,trade_in_2h_usdtfutures_ADAUSDT_close_price,trade_in_1h_usdtfutures_ADAUSDT_close_price,usdtfutures_ADAUSDT_close_price,usdtfutures_ADAUSDT_volume,SMA(usdtfutures_ADAUSDT_close_price_5),SMA(usdtfutures_ADAUSDT_volume_5),SMA(usdtfutures_ADAUSDT_close_price_60),shift1_SMA(usdtfutures_ADAUSDT_close_price_60),shift2_SMA(usdtfutures_ADAUSDT_close_price_60),...,shift8_SMA(usdtfutures_ADAUSDT_close_price_240),SMA(usdtfutures_ADAUSDT_volume_240),shift1_SMA(usdtfutures_ADAUSDT_volume_240),shift2_SMA(usdtfutures_ADAUSDT_volume_240),shift3_SMA(usdtfutures_ADAUSDT_volume_240),shift4_SMA(usdtfutures_ADAUSDT_volume_240),shift5_SMA(usdtfutures_ADAUSDT_volume_240),shift6_SMA(usdtfutures_ADAUSDT_volume_240),shift7_SMA(usdtfutures_ADAUSDT_volume_240),shift8_SMA(usdtfutures_ADAUSDT_volume_240)
2020-05-29 04:30:59.999000+00:00,-0.014422,-0.008502,-0.013056,-0.001213,0.262093,-0.000848,-0.095193,0.000468,-0.000044,0.000066,...,NaN,0.928105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-29 04:31:59.999000+00:00,-0.013823,-0.008051,-0.011545,-0.000607,2.041941,-0.000546,0.416770,0.000455,-0.000080,0.000035,...,NaN,0.930961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-29 04:32:59.999000+00:00,-0.017860,-0.012108,-0.015135,0.003646,-0.546090,0.000426,0.176419,0.000500,-0.000093,0.000086,...,NaN,0.932296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-29 04:33:59.999000+00:00,-0.016821,-0.010911,-0.014093,-0.001211,0.653995,0.000214,0.474442,0.000480,-0.000093,0.000058,...,NaN,0.895109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-29 04:34:59.999000+00:00,-0.017270,-0.009696,-0.014392,0.000303,-0.742293,0.000184,0.333929,0.000513,-0.000145,0.000078,...,NaN,0.894472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-28 17:56:59.999000+00:00,-0.010607,0.013291,0.022720,-0.005211,0.702292,-0.002260,0.279952,0.000161,-0.000368,-0.000147,...,0.000093,0.094517,0.087923,0.100281,0.136143,0.154483,0.149851,0.215798,0.183478,0.120457
2021-05-28 17:57:59.999000+00:00,-0.011662,0.010286,0.022669,-0.000655,-0.308740,-0.002482,0.255626,0.000143,-0.000337,-0.000083,...,0.000092,0.094669,0.083033,0.103878,0.137120,0.153567,0.149852,0.205928,0.193431,0.118644
2021-05-28 17:58:59.999000+00:00,-0.016259,0.009272,0.016389,0.003407,-0.228650,-0.001684,0.272110,0.000256,-0.000376,-0.000019,...,0.000081,0.093493,0.084805,0.103351,0.137377,0.153505,0.147421,0.208986,0.191580,0.120004
2021-05-28 17:59:59.999000+00:00,-0.019332,0.004947,0.014190,0.003134,0.138020,-0.000927,0.237445,0.000320,-0.000404,-0.000035,...,0.000095,0.092522,0.084291,0.103066,0.132710,0.160777,0.149299,0.208263,0.187289,0.122769


In [3]:
def check_x_for_leak(X):
    for column in X.columns:
        assert "trade_in_" not in column

def create_model(df_orig, index, train_size, test_size, coin):
    df_orig = df_orig.fillna(0)
    del df_orig["trade_in_3h_usdtfutures_{}USDT_close_price".format(coin)]
    del df_orig["trade_in_2h_usdtfutures_{}USDT_close_price".format(coin)]
    start = index - (train_size+test_size)
    end = index
    df_orig = df_orig[start:end]

    df = df_orig.copy()
    df = df.sample(frac=1/2)
    Y = df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)]

    Y_MEAN = np.mean(np.abs(Y))
    Y_MEAN = Y_MEAN * (1/4)

    Y[df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)] > Y_MEAN] = 20
    Y[df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)] < -Y_MEAN] = 10
    Y[df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)] < 9] = 1

    first_element = "usdtfutures_{}USDT_close_price".format(coin)
    #first_element = "usdtfutures_{}USDT_close_price_ewm_mean".format(coin)
    
    X = df.loc[:, first_element :]
    check_x_for_leak(X)

    test_size = test_size / (test_size+train_size)
    test_size = test_size or 0.01
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=test_size, random_state=6, shuffle=False
    )
    hours_in_test = len(Y_test)

    model = XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        #booster=["gbtree","gblinear"][0],
        gamma=0.1,
        #sampling_method=["uniform", "gradient_based"][1],
        #num_parallel_tree=1,
        #reg_alpha=1,
        #reg_lambda=0,
        #eta=0.3,
    )
    model.fit(X_train, Y_train)
    return model

In [4]:
def predict_row(df_orig, index, model, coin):
    df_orig = df_orig.fillna(0)
    del df_orig["trade_in_3h_usdtfutures_{}USDT_close_price".format(coin)]
    del df_orig["trade_in_2h_usdtfutures_{}USDT_close_price".format(coin)]
    df_orig = df_orig.iloc[index:index+1]
    
    first_element = "usdtfutures_{}USDT_close_price".format(coin)
    #first_element = "usdtfutures_{}USDT_close_price_ewm_mean".format(coin)
    X = df_orig.loc[:,first_element :]
    check_x_for_leak(X)
    y_pred = model.predict(X)[0]
    Y_TEST_REAL = df_orig["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)][0]
    
    profit = -1
    
    side = -1
    if y_pred == 20:
        side = 1
    if y_pred == 1:
        return Y_TEST_REAL, 0

    if side * Y_TEST_REAL > 0:
        profit = 1
    
    return abs(Y_TEST_REAL) * profit, side
    
    


In [5]:
df_orig_A = pd.read_hdf("v2_lagging_A28may_28may_2020_2021_ada.hdf")
last_model_train_A=0
retrain_limit_A = 60*4
res = []
model_A = None
for i in tqdm.tqdm(range(60*24*340, 60*24*362, 60)):
    if model_A is None:
        model_A = create_model(df_orig_A.copy(), i, (60*24*1), 60, "ADA")
        
    predictA = predict_row(df_orig_A, i, model_A, "ADA")
    res.append(predictA)
    if (i - last_model_train_A) > retrain_limit_A:
        model_A = None
        

  0%|          | 0/528 [00:00<?, ?it/s]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:36:08] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
  0%|          | 1/528 [00:00<05:17,  1.66it/s]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:36:08] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
  0%|          | 2/528 [00:01<04:48,  1.82it/s]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
100%|█████████▉| 526/528 [05:20<00:01,  1.65it/s]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:41:28] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
100%|█████████▉| 527/528 [05:21<00:00,  1.60it/s]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:41:29] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
100%|██████████| 528/528 [05:21<00:00,  1.64it/s]


In [6]:
[x[0] for x in res if x[1] !=0]

[0.0025292063109719765,
 0.003026141028737106,
 0.015857877088206185,
 0.016474895397489586,
 0.0063181948879371674,
 0.007998265796574922,
 -0.003134678130345403,
 0.019087323943662082,
 -0.008783618207437227,
 -0.01419223251465962,
 -0.006269592476489061,
 0.011573208722741368,
 0.0037648399365597385,
 0.0034515819750718935,
 -0.007842986338272073,
 -0.017373839198436047,
 -0.010903947337742803,
 0.005343194538409479,
 0.015195900353231151,
 0.005931035532917628,
 0.0020284282708405987,
 0.00029578169793853526,
 -0.0058683933188264925,
 -0.003027783921475957,
 0.009822371138752395,
 0.017855249003833285,
 0.0032895960941629054,
 -0.005653662994138164,
 -0.013105404686622801,
 0.006693987078709052,
 0.008876901154506012,
 0.03776059494555706,
 -0.009902158892847068,
 0.007633962369963276,
 0.027396497867692604,
 0.01730103806228367,
 0.022994129158512733,
 0.01021817177575246,
 -0.00990173127177707,
 -0.0043989579968244374,
 0.01902671057446037,
 0.003102875918837511,
 0.0377853439077

In [7]:
sum([x[0] for x in res if x[1] !=0])

4.157236345094074